# 1. Data Set 설명

# 2. Data Preparation

In [1]:
# 라이브러리

from jbfg.pandas_utils import *
import calendar
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import numpy as np 
import pandas as pd
import random
import IPython.display

color = sns.color_palette("crest", 8)
cmap_colors = sns.cubehelix_palette(start=.5, rot=-.5, as_cmap=True)

%matplotlib inline


pd.options.mode.chained_assignment = None

In [75]:
start, end = 2, 20
cluster_model = "kmeans"
column_level = "department"
PCA_mode = False
if PCA_mode :
    n_components = int(input("PCA에 사용할 n_components 개수를 입력하세요 : "))
    
quick_test = True
K = list(range(start, end+1))

In [2]:
# 테이블 조회

schema = 'l2_kjbk_extnl'
products_sql = 'select * from products_info'
train_x_past_sql = 'select * from train_x_past'
train_x_current_sql = 'select * from train_x_current'
train_y_sql = 'select * from train_y'
test_x_past_sql = 'select * from test_x_past'
test_x_current_sql = 'select * from test_x_current'

In [3]:
df_product = get_df(products_sql, schema)
df_train_past = get_df(train_x_past_sql, schema)
df_train_curr = get_df(train_x_current_sql, schema)
df_train_y = get_df(train_y_sql, schema)
df_test_past =(test_x_past_sql, schema)
df_test_curr =(test_x_current_sql, schema)

In [15]:
display(df_)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
0,717029,196613,15,1,9,3.0,5,0,41220
1,2325772,196613,35,6,11,2.0,7,1,41220
2,642668,196613,69,2,16,2.0,11,1,4210
3,1326515,196613,61,3,19,3.0,3,1,4210
4,1437845,196613,36,3,15,4.0,16,0,4210
...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852
1253968,1969590,163819,42,1,15,6.0,3,1,24852
1253969,2549215,163819,9,1,11,7.0,1,1,24852
1253970,3103211,163819,39,1,14,7.0,3,1,24852


In [ ]:
# 테이블 합치기

In [4]:
# train_c = train_x_current + train_y 

df_train_curr = df_train_curr.reset_index()
df_train_y = df_train_y.reset_index()
df_train_c = pd.merge(df_train_curr, df_train_y,on='index', how='left')
df_train_c.drop('index', axis = 1)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
0,806422,41116,47,5,18,3.0,4,1,21709
1,2146033,127113,45,1,10,7.0,24,1,21709
2,3055498,140168,45,5,10,26.0,11,1,21709
3,3267527,60604,52,0,16,8.0,2,1,21709
4,3125992,136554,46,4,9,5.0,2,0,21709
...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,24852
24420,1754722,6710,100,5,11,3.0,21,1,24852
24421,1076973,150840,100,0,13,0.0,18,1,24852
24422,1247049,99707,100,1,13,3.0,6,1,24852


In [50]:
# null값 확인
df_train['eval'].isnull().sum()
# 결측치 채우기 : fillna() : df['col'].fillna('Nan')

0

In [89]:
# trp = train_past + product

df_trp = pd.merge(df_train_past, df_product, on='product_id', how='left')
display(df_trp)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name
0,717029,196613,15,1,9,3.0,5,0,41220,Organic Romaine Lettuce,83,4,fresh vegetables,produce
1,2325772,196613,35,6,11,2.0,7,1,41220,Organic Romaine Lettuce,83,4,fresh vegetables,produce
2,642668,196613,69,2,16,2.0,11,1,4210,Whole Milk,84,16,milk,dairy eggs
3,1326515,196613,61,3,19,3.0,3,1,4210,Whole Milk,84,16,milk,dairy eggs
4,1437845,196613,36,3,15,4.0,16,0,4210,Whole Milk,84,16,milk,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253967,1902783,163819,47,0,19,13.0,2,1,24852,Banana,24,4,fresh fruits,produce
1253968,1969590,163819,42,1,15,6.0,3,1,24852,Banana,24,4,fresh fruits,produce
1253969,2549215,163819,9,1,11,7.0,1,1,24852,Banana,24,4,fresh fruits,produce
1253970,3103211,163819,39,1,14,7.0,3,1,24852,Banana,24,4,fresh fruits,produce


In [90]:
# trc = train_c + product

df_trc = pd.merge(df_train_c, df_product, on='product_id', how='left')
display(df_trc)

,index,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name
0,0,806422,41116,47,5,18,3.0,4,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
1,1,2146033,127113,45,1,10,7.0,24,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
2,2,3055498,140168,45,5,10,26.0,11,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
3,3,3267527,60604,52,0,16,8.0,2,1,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
4,4,3125992,136554,46,4,9,5.0,2,0,21709,Sparkling Lemon Water,115,7,water seltzer sparkling water,beverages
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,24419,3184000,174046,100,5,7,2.0,12,1,24852,Banana,24,4,fresh fruits,produce
24420,24420,1754722,6710,100,5,11,3.0,21,1,24852,Banana,24,4,fresh fruits,produce
24421,24421,1076973,150840,100,0,13,0.0,18,1,24852,Banana,24,4,fresh fruits,produce
24422,24422,1247049,99707,100,1,13,3.0,6,1,24852,Banana,24,4,fresh fruits,produce


In [91]:
# train = train_past + train_current

df_train = pd.concat([df_train_past, df_train_c], axis=0)
df_train['eval'] = np.where(df_train['index'] >= 0 , "current", "past")
display(df_train)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,index,eval
0,717029,196613,15,1,9,3.0,5,0,41220,NaN,past
1,2325772,196613,35,6,11,2.0,7,1,41220,NaN,past
2,642668,196613,69,2,16,2.0,11,1,4210,NaN,past
3,1326515,196613,61,3,19,3.0,3,1,4210,NaN,past
4,1437845,196613,36,3,15,4.0,16,0,4210,NaN,past
...,...,...,...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1,24852,24419.0,current
24420,1754722,6710,100,5,11,3.0,21,1,24852,24420.0,current
24421,1076973,150840,100,0,13,0.0,18,1,24852,24421.0,current
24422,1247049,99707,100,1,13,3.0,6,1,24852,24422.0,current


In [92]:
# tr = train + product

df_tr = pd.merge(df_train, df_product, on='product_id', how='left')
display(df_tr)

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,index,eval,name,category_id,division_id,category_name,division_name
0,717029,196613,15,1,9,3.0,5,0,41220,NaN,past,Organic Romaine Lettuce,83,4,fresh vegetables,produce
1,2325772,196613,35,6,11,2.0,7,1,41220,NaN,past,Organic Romaine Lettuce,83,4,fresh vegetables,produce
2,642668,196613,69,2,16,2.0,11,1,4210,NaN,past,Whole Milk,84,16,milk,dairy eggs
3,1326515,196613,61,3,19,3.0,3,1,4210,NaN,past,Whole Milk,84,16,milk,dairy eggs
4,1437845,196613,36,3,15,4.0,16,0,4210,NaN,past,Whole Milk,84,16,milk,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278391,3184000,174046,100,5,7,2.0,12,1,24852,24419.0,current,Banana,24,4,fresh fruits,produce
1278392,1754722,6710,100,5,11,3.0,21,1,24852,24420.0,current,Banana,24,4,fresh fruits,produce
1278393,1076973,150840,100,0,13,0.0,18,1,24852,24421.0,current,Banana,24,4,fresh fruits,produce
1278394,1247049,99707,100,1,13,3.0,6,1,24852,24422.0,current,Banana,24,4,fresh fruits,produce


# 3. EDA 및 데이터 전처리

In [52]:
# 상품 갯수
print("Unique products : {}".format(len(df_product.product_id.unique())))

Unique products : 100


In [71]:
# train 고객수
print("Unique cstnos : {}".format(len(df_tr.cstno.unique())))

Unique cstnos : 5859


In [72]:
print(df_tr.shape)

(1278396, 16)


In [111]:
print("과거 가장 많이 구매한 상품 10개")    
if quick_test :
    display(df_trp.name.value_counts()[:10]) 

print("\n과거 거래내역에 가장 많이 포함된 카테고리 10개 ")   
if quick_test :
    display(df_trp.category_name.value_counts()[:10])

print("\n과거 거래내역에 가장 많이 포함된 담당부서 10개")  
if quick_test :
    display(df_trp.division_name.value_counts()[:10])

과거 가장 많이 구매한 상품 10개


Banana                    74686
Bag of Organic Bananas    67514
Organic Strawberries      52917
Organic Hass Avocado      42604
Organic Baby Spinach      40112
Organic Whole Milk        29757
Organic Raspberries       28207
Organic Avocado           25046
Limes                     21216
Large Lemon               21012
Name: name, dtype: int64


과거 거래내역에 가장 많이 포함된 카테고리 10개 


fresh fruits                     501780
fresh vegetables                 282838
packaged vegetables fruits       149855
milk                              57404
water seltzer sparkling water     41593
packaged cheese                   35362
fresh herbs                       26949
cream                             26499
soy lactosefree                   23722
packaged produce                  13583
Name: category_name, dtype: int64


과거 거래내역에 가장 많이 포함된 담당부서 10개


produce         975005
dairy eggs      166335
beverages        52044
deli             19586
frozen           11506
bakery            9083
pantry            7436
canned goods      6692
meat seafood      6285
Name: division_name, dtype: int64

In [112]:
print("현재 가장 많이 구매한 상품 10개")    
if quick_test :
    display(df_trc.name.value_counts()[:10]) 

print("\n현재 거래내역에 가장 많이 포함된 카테고리 10개 ")   
if quick_test :
    display(df_trc.category_name.value_counts()[:10])

print("\n현재 거래내역에 가장 많이 포함된 담당부서 10개")  
if quick_test :
    display(df_trc.division_name.value_counts()[:10])

현재 가장 많이 구매한 상품 10개


Bag of Organic Bananas    1435
Banana                    1340
Organic Strawberries      1115
Organic Hass Avocado       694
Organic Baby Spinach       683
Organic Raspberries        577
Organic Whole Milk         517
Organic Avocado            514
Large Lemon                471
Organic Cucumber           444
Name: name, dtype: int64


현재 거래내역에 가장 많이 포함된 카테고리 10개 


fresh fruits                     9846
fresh vegetables                 5499
packaged vegetables fruits       3046
milk                              973
water seltzer sparkling water     793
packaged cheese                   612
fresh herbs                       557
cream                             478
soy lactosefree                   459
packaged produce                  383
Name: category_name, dtype: int64


현재 거래내역에 가장 많이 포함된 담당부서 10개


produce         19331
dairy eggs       2978
beverages         972
deli              391
frozen            229
bakery            164
canned goods      144
pantry            118
meat seafood       97
Name: division_name, dtype: int64

In [113]:
print("\n과거 가장 많은 상품 갯수를 구매한 고객 (top10)")
if quick_test :
    display(df_trp.cstno.value_counts()[:10])

print("\n현재 가장 많은 상품 갯수를 구매한 고객 (top10)")
if quick_test :
    display(df_trc.cstno.value_counts()[:10])


과거 가장 많은 상품 갯수를 구매한 고객 (top10)


150840    795
184451    766
103593    762
100787    751
54485     745
115495    736
95967     734
172806    733
175294    728
160106    727
Name: cstno, dtype: int64


현재 가장 많은 상품 갯수를 구매한 고객 (top10)


148812    24
185920    17
21451     17
150100    17
103634    17
102282    17
93471     16
138401    16
72323     16
162875    16
Name: cstno, dtype: int64

In [ ]:
# <<데이터 전처리 작업>> 
# 결측치 제거, 정규화
# 고객 단위로 어떤 상품을 구매했는지 정보만 가지는 feature vector로 변환 (crosstab,CountVectorizer)

In [128]:
columns = ["name", "cstno", "category_name", "division_name"]
columns = np.array(columns)
columns = np.setdiff1d(df_trp.columns, columns)
df_trp.drop(columns=columns, inplace=True)

In [129]:
if columns == "division_name" : cstno_matrix = pd.crosstab(df_trp.cstno, df_trp.division_name)
elif columns == "category_name" : cstno_matrix = pd.crosstab(df_trp.cstno, df_trp.category_name)
else : cstno_matrix = pd.crosstab(df_trp.cstno, df_trp.name)
cstno_matrix

/opt/conda/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  


name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
cstno,,,,,,,,,,,,,,,,,,,,,
10000,0,41,0,0,0,6,30,0,0,0,...,0,1,1,0,2,0,0,0,1,1
100001,20,0,0,4,0,41,0,0,3,0,...,3,0,0,0,0,0,0,0,0,1
100046,0,0,0,0,1,2,21,3,0,0,...,0,2,0,2,0,0,0,0,0,0
100071,0,0,0,0,0,6,0,0,0,0,...,0,0,4,0,0,0,2,0,0,0
100108,0,0,0,14,0,20,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,0,0,15,0,0,0,45,0,0,0,...,0,6,0,0,0,0,0,0,0,0
99829,0,1,0,0,0,1,34,0,8,1,...,0,0,0,0,1,1,0,16,0,12
99887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_tr[df_tr.cstno == '196613' ]

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,index,eval,name,category_id,division_id,category_name,division_name
0,717029,196613,15,1,9,3.0,5,0,41220,NaN,past,Organic Romaine Lettuce,83,4,fresh vegetables,produce
1,2325772,196613,35,6,11,2.0,7,1,41220,NaN,past,Organic Romaine Lettuce,83,4,fresh vegetables,produce
2,642668,196613,69,2,16,2.0,11,1,4210,NaN,past,Whole Milk,84,16,milk,dairy eggs
3,1326515,196613,61,3,19,3.0,3,1,4210,NaN,past,Whole Milk,84,16,milk,dairy eggs
4,1437845,196613,36,3,15,4.0,16,0,4210,NaN,past,Whole Milk,84,16,milk,dairy eggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2978591,196613,82,6,17,0.0,17,1,24852,NaN,past,Banana,24,4,fresh fruits,produce
201,3193705,196613,55,3,17,0.0,12,1,24852,NaN,past,Banana,24,4,fresh fruits,produce
202,3329257,196613,42,6,14,3.0,10,1,24852,NaN,past,Banana,24,4,fresh fruits,produce
1255604,2414402,196613,100,6,13,5.0,4,1,43122,1632.0,current,Organic Bartlett Pear,24,4,fresh fruits,produce


In [27]:
evl_cnt = df_train_past.evl_set.value_counts()

plt.figure(figsize=(8,6))
sns.barplot(evl_cnt.index, evl_cnt.values, alpha = 0.9, color = color[0])
plt.ylabel('Number of orders', fontsize = 13)
plt.xlabel('Eval set type', fontsize = 13)
plt.show()


AttributeError: 'DataFrame' object has no attribute 'evl_set'

In [3]:
df.to_pickle('./train_y.pickle')

In [5]:
y1 = pd.read_pickle('./train_y.pickle')

In [ ]:
#예시1
df.to_csv(‘./output파일명.csv’, encoding='CP949')


#예시2
df.to_csv(f'./output파일명_{가변변수명}.csv’, encoding='CP949')